# Initializations

In [1]:
import numpy as np
import pandas as pd

import sys, os, time
import glob

from matplotlib import pyplot as plt
%matplotlib inline

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

# widgets and interaction
from ipywidgets import FloatProgress
from IPython.display import display, clear_output

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from skimage.io import imread, imsave

import warnings
warnings.filterwarnings('ignore')

import gzip
import cPickle as pickle

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.utils as vutils
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

import sys
sys.path.append("../ml-model/")

import models.dcgan as dcgan
import models.mlp as mlp
import models.dcgan_orig as do

# Set up data sources

In [3]:
import re
def parse_list(s):
    s = re.sub('\s+', ' ', s)[2:-2]
    return tuple([float(n.strip()) for n in s.split(" ")])

In [4]:
files_df = pd.read_csv("/home/data/world-cities/urban_areas_over_10kpop_stats.csv")
files_df['patch distr 448'] = files_df['patch distr 448'].apply(parse_list)
files_df['patch distr 128'] = files_df['patch distr 128'].apply(parse_list)
files_df['patch distr 64'] = files_df['patch distr 64'].apply(parse_list)

files_df.head()

,Unnamed: 0,filename,class,country,population,city,phase,region,build pct 128,fractal dim 128,patch distr 128,build pct 448,fractal dim 448,patch distr 448,build pct 64,fractal dim 64,patch distr 64,decile
0,0,/home/data/world-cities//train/medium/SAR_in_i...,medium,in,100585,"itarsi, in (pop 100.6k)",train,Asia,0.011440,1.802166,"(0.0, 0.69314718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.011681,1.559993,"(3.17805383, 1.79175947, 2.30258509, 1.0986122...",0.012975,1.511785,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3
1,1,/home/data/world-cities//train/medium/SAR_co_s...,medium,co,126553,"sogamoso, co (pop 126.6k)",train,Americas,0.003207,1.409835,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.003182,1.081987,"(2.56494936, 1.60943791, 1.79175947, 0.0, 0.0,...",0.002289,0.862936,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2
2,2,/home/data/world-cities//train/medium/SAR_us_l...,medium,us,140772,"lakewood, us (pop 140.8k)",train,Americas,0.027446,1.422604,"(0.0, 0.0, 0.0, 0.0, 0.69314718, 0.0, 0.0, 0.0...",0.027316,1.254302,"(3.55534806, 2.63905733, 1.79175947, 1.3862943...",0.026681,1.322882,"(0.0, 1.09861229, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",5
3,3,/home/data/world-cities//train/medium/SAR_br_t...,medium,br,251647,"taubate, br (pop 251.6k)",train,Americas,0.038727,1.711233,"(1.79175947, 0.69314718, 1.60943791, 0.0, 0.0,...",0.038346,1.464830,"(4.33073334, 3.49650756, 3.25809654, 2.9957322...",0.038245,1.561612,"(1.09861229, 0.69314718, 0.0, 0.0, 0.0, 0.0, 0...",6
4,4,/home/data/world-cities//train/medium/SAR_ve_m...,medium,ve,105511,"mariara, ve (pop 105.5k)",train,Americas,0.030704,1.629995,"(1.38629436, 0.0, 1.38629436, 0.0, 0.0, 0.0, 0...",0.031249,1.415175,"(3.98898405, 3.04452244, 2.7080502, 2.19722458...",0.032483,1.485818,"(0.69314718, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",6


In [5]:
idx = np.random.choice(range(len(files_df)), size=int(0.8*len(files_df)), replace=False)
train_df = files_df.iloc[idx]
valid_df = files_df.iloc[list(set(range(len(files_df)))-set(idx))]

print len(train_df), len(valid_df)

19988 4998


In [7]:
train_df.to_csv("/home/data/world-cities/" + "train.csv")
valid_df.to_csv("/home/data/world-cities/" + "valid.csv")

#### Set up batching

In [8]:
pxCrop    = 448
classCol  = "patch distr 128"
imageSize = 128
batchSize = 64
workers   = 4

In [33]:
import sys
sys.path.append("./../ml-model/pytorch_utils")
from loader_dataframe import ImageDataFrame, grayscale_loader, WeightedRandomSampler

from PIL import Image

def fn_rotate(img, max_angle=30):
    theta = np.random.randn()*max_angle
    return img.rotate(theta, expand=True)

train_dataset = ImageDataFrame(df=train_df, 
                         classCol=classCol,
                         loader=grayscale_loader,
                         transform=transforms.Compose([
                               transforms.RandomHorizontalFlip(),
                               transforms.CenterCrop(pxCrop),
                               transforms.Lambda(lambda img: fn_rotate(img)),
                               transforms.Scale(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0, 0, 0), (255.0, 255.0, 255.0))
                           ]))
valid_dataset = ImageDataFrame(df=valid_df, 
                         classCol=classCol,
                         loader=grayscale_loader,
                         transform=transforms.Compose([
                               transforms.CenterCrop(pxCrop),
                               transforms.Scale(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0, 0, 0), (255.0, 255.0, 255.0))

                           ]))

list_output = train_df[classCol].dtype == object
if list_output:
    n_classes = len(train_df[classCol].iloc[0])
else:
    n_classes = len(train_dataset.classes) if train_dataset.classes is not None else None

weights = None #train_df['built pct']
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize,
                                         shuffle=True, 
                                         #sampler=WeightedRandomSampler(weights, len(train_df)),
                                         num_workers=int(workers))

valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batchSize,
                                         shuffle=False, 
                                         num_workers=int(workers))

# Define model
- should work on arbitrary size images (multiple of 16)
- dual output: 0/1 for fake/real, vector of stats

In [54]:
nz = 100
nc = 1
ndf= 16
ngpu = 4
lr = 0.00005
n_extra_layers = 0
beta1 = 0.5
lam = torch.FloatTensor(n_classes).fill_(0.5)

In [56]:
import torch
import torch.nn as nn
import torch.nn.parallel

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class DCGAN_D_DUAL(nn.Module):
    def __init__(self, isize, nz, nc, ndf, ngpu, n_extra_layers=0, n_classes=None):
        super(DCGAN_D_DUAL, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        features = nn.Sequential()
        # input is nc x isize x isize
        features.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        features.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            features.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            features.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            features.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            features.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            features.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            features.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        features.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, cndf, 4, 1, 0, bias=False))
        self.features = features
        self.flat_fts = self.get_flat_fts((nc,isize,isize), self.features)

        self.classifier_src = nn.Sequential(
            nn.Linear(self.flat_fts, 1),
#             nn.Dropout(p=0.2),
#             nn.ReLU(),
#             nn.Linear(100,1),
            nn.Sigmoid()
        )
        if n_classes is None:
            n_classes = 1
        self.classifier_cls = nn.Sequential(
            nn.Linear(self.flat_fts, n_classes),
#             nn.Dropout(p=0.2),
             nn.ReLU(),
#             nn.Linear(100,n_classes),
#             nn.Softmax()
        )
        
    def get_flat_fts(self, in_size, fts):
        f = fts(Variable(torch.ones(1,*in_size)))
        return int(np.prod(f.size()[1:]))
    
    def forward(self, x):
        fts = self.features(x)
        flat_fts = fts.view(-1, self.flat_fts)
        out1 = self.classifier1(flat_fts)
        out2 = self.classifier2(flat_fts)
        return out1, out2
    
    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            fts = nn.parallel.data_parallel(self.features, input, range(self.ngpu))
            flat_fts = fts.view(-1, self.flat_fts)
            out_src= nn.parallel.data_parallel(self.classifier_src, flat_fts, range(self.ngpu))
            out_cls= nn.parallel.data_parallel(self.classifier_cls, flat_fts, range(self.ngpu))            
        else: 
            fts = self.features(input)
            out_src= self.classifier_src(fts)
            out_cls= self.classifier_cls(fts)
                
        return out_src, out_cls


# Train model

In [71]:
net = DCGAN_D_DUAL(imageSize, nz, nc, ndf, ngpu, n_extra_layers, n_classes=n_classes)
net.apply(weights_init)
    
input = torch.FloatTensor(batchSize, nc, imageSize, imageSize)
if n_classes is not None:
    if not list_output:
        label = torch.LongTensor(batchSize, 1)
    else:
        label = torch.FloatTensor(batchSize, n_classes)
else:
    label = torch.FloatTensor(batchSize, 1)
    
source_label = torch.FloatTensor(batchSize)

lam = Variable(lam.cuda())

if torch.cuda.is_available():
    net.cuda()
    input = input.cuda()
    label, source_label = label.cuda(), source_label.cuda()

input = Variable(input)
label = Variable(label, requires_grad=False)
source_label = Variable(source_label, requires_grad=False)

# setup optimizer
optimizer = optim.Adam(net.parameters(), lr=lr, betas=(beta1, 0.999))

# optimization criteria
criterion_bce = nn.BCELoss()
criterion_mse = nn.MSELoss()
criterion_cls = nn.CrossEntropyLoss()


In [58]:
# def compute_eval_performance(net, valid_dataloader):
#     correct = 0
#     total = 0
#     for imgs, labs in valid_dataloader:
#         imgs = Variable(imgs.cuda())
#         labs = labs.cuda()
#         out_src_test, out_sts_test = net(imgs)
#         loss_src_test = criterion_bce(out_src_test, source_label)
#         loss_sts_test = criterion_cls(out_sts_test, labs)
#         _, predicted = torch.max(out_sts_test.data, 1)
#         total += labs.size(0)
#         correct += (predicted == labs).sum()
    
#     val_acc = correct / float(total)


In [59]:
num_epochs = 25

save_dir = "/home/data/pytorch-workspace/analyzer/classifier/"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

train_loss_hist = []
valid_loss_hist = []
best_val_perf  = 0

for epoch in range(num_epochs):
    
    # Train for this epoch
    # --------------------
    
    net.train(True)
    train_loss_epoch = []
    for i, (imgs, labs) in enumerate(train_dataloader):    
        labs = torch.cat(labs, 1)
        batch_size = imgs.size(0)
        input.data.resize_(imgs.size()).copy_(imgs)
        label.data.resize_(labs.size()).copy_(labs)
        source_label.data.resize_(batch_size).fill_(1) # right now, all images are real

        # Forward + Backward + Optimize
        net.zero_grad()
        out_src, out_sts = net(input)
        loss_src = criterion_bce(out_src, source_label)
        loss_sts = criterion_mse(out_sts * lam, label * lam) # lam can be a vector of weights
        # loss = 0*loss_src + lam * loss_sts    
        loss = loss_sts
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f (Source: %.4f, Stats: %.4f)' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, 
                     loss.data[0], loss_src.data[0], loss_sts.data[0]*lam))
        train_loss_epoch.append((loss.data[0], loss_src.data[0], loss_sts.data[0]*lam))
    
    train_loss_epoch = np.array(train_loss_epoch).mean(0)
    train_loss_hist.append(train_loss_epoch)
    
    # Test for this epoch
    # -------------------

    net.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    valid_loss_epoch = []
    for imgs, labs in valid_dataloader:
        labs = torch.cat(labs, 1)
        imgs = Variable(imgs.cuda())
        labs = Variable(labs.float().cuda())
        src_labs = Variable(torch.FloatTensor(labs.data.size(0)).cuda(), requires_grad=False)
        out_src_valid, out_sts_valid = net(imgs)
        loss_src_valid = criterion_bce(out_src_valid, src_labs)
        loss_sts_valid = criterion_mse(out_sts_valid * lam, labs * lam)
        loss_valid = loss_sts # + loss_src
        valid_loss_epoch.append((loss_valid.data[0], loss_src_valid.data[0], loss_sts_valid.data[0]))   
        
    valid_loss_epoch = np.array(valid_loss_epoch).mean(0)
    valid_loss_hist.append(valid_loss_epoch)
    
    # Track performance
    # -----------------

    clear_output(wait=True)
    
    # plot performance vs epoch
    fig, ax = plt.subplots(1,2, figsize=(8,3))
    ax[0].plot(range(len(train_loss_hist)), 
             [x[0] for x in train_loss_hist],label="loss (train)")
    ax[0].plot(range(len(train_loss_hist)), 
             [x[2] for x in train_loss_hist],label="stat loss (train)")
    ax[0].legend(loc="best")
    ax[1].plot(range(len(valid_loss_hist)), 
             [x[0] for x in valid_loss_hist],label="loss (valid)")
    ax[1].plot(range(len(valid_loss_hist)), 
             [x[2] for x in valid_loss_hist],label="stat loss (valid)")
    ax[1].legend(loc="best")
    plt.suptitle("Analyzer model performance")
    plt.savefig("%s/training_progress.jpg"%save_dir)
    plt.show()
    plt.close(fig)

#     # checkpoint best model
#     if val_acc > best_val_acc:
#         print "checkpointing: validation accuracy improved from %.2f to %.2f"%(best_val_acc, val_acc)
#         torch.save(net.state_dict(), '{0}/net_epoch{1}_acc{2:.2f}.pth'.format(save_dir, epoch, val_acc))
#         best_val_acc = val_acc
        

AssertionError: 

In [62]:
out_sts.size(), lam.size()

(torch.Size([64, 19]), torch.Size([19]))

In [73]:
torch.sq(((out_sts - label)**2)).dot(lam)

RuntimeError: sizes do not match at /data/users/soumith/builder/wheel/pytorch-src/torch/lib/THC/generic/THCTensorMathBlas.cu:11

In [74]:
lam

Variable containing:
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
 0.5000
[torch.cuda.FloatTensor of size 19 (GPU 0)]

In [ ]:
plt.imshow(imread("%s/training_progress.jpg"%save_dir))